In [4]:
import os, sys
from tabulate import tabulate
from tqdm import tqdm
import pandas as pd
from z3 import set_param

from analysis.singleton_analyzer import SingletonAnalyzer
from base.exper import Experiment
from base.exper_analyzer import ExperAnalyzer
from debugger3 import Debugger3
from utils.system_utils import log_info
from benchmark_consts import *
from debugger.informed_editor import InformedEditor

os.chdir("/home/amarshah/mariposa_fixed/mariposa/")
from base.factory import FACT

SOLVER = FACT.get_solver("z3_4_13_0")
VERI_CFG = FACT.get_config("verify")
FILTER_CFG = FACT.get_config("verus_quick")
QA = FACT.get_analyzer("10sec")


def get_debug_status(dbg: Debugger3):
    try:
        p_singleton = FACT.get_project_by_path(dbg.singleton_dir)
    except:
        return "singleton not created"

    e_singleton = FACT.try_get_exper(p_singleton, VERI_CFG, SOLVER)

    if e_singleton is None:
        return "singleton not ran"

    try:
        p_filter = FACT.get_project_by_path(dbg.singleton_filtered_dir)
    except:
        return "singleton not filtered"

    e_filter = FACT.try_get_exper(p_filter, FILTER_CFG, SOLVER)

    if e_filter is None:
        return "filtered but not ran"

    return e_filter

current_dbgs = dict()

for query in tqdm(UNSTABLE_10_SECS):
    if query in SKIP_FOR_NOW:
        continue

    dbg = Debugger3(query)
    current_dbgs[query] = dbg
    assert dbg.editor is not None


  0%|          | 0/70 [00:00<?, ?it/s]

[DEBUG] [edit] proof path: dbg/199ba4594c/proofs/reseed.3199813507728074554.proof 
[DEBUG] [edit] trace path: dbg/199ba4594c/traces/reseed.6506362136541185908 


[INFO] [init] query path: data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2 
[INFO] [init] dbg root: dbg/199ba4594c 


[DEBUG] [term graph] 64997 nodes, 234175 edges, root @[h!392b5117] 
[DEBUG] 41 nodes are unreachable! 
  0%|          | 0/70 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
table = []

for query, dbg in current_dbgs.items():
    p_singleton = FACT.get_project_by_path(dbg.singleton_dir)
    e_singleton = FACT.try_get_exper(p_singleton, VERI_CFG, SOLVER)
    ba = SingletonAnalyzer(e_singleton, QA)
    editor: InformedEditor = dbg.editor

    root_quants = editor.get_singleton_actions(skip_infeasible=False)
    root_quant_count = len(root_quants)
    tested_quant_count = len(dbg.get_edited_qnames(ba.qids))
    assert tested_quant_count <= root_quant_count
    tested_edit_count = len(ba.qids)

    p_filter = FACT.get_project_by_path(dbg.singleton_filtered_dir)
    e_filter = FACT.try_get_exper(p_filter, FILTER_CFG, SOLVER)
    fa = ExperAnalyzer(e_filter, QA)
    stabilized_eids = []
    
    for eid in fa.get_stable_edit_ids():
        ei = dbg.look_up_edit_with_id(eid)
        qname, action = ei.get_singleton_edit()
        if qname == "prelude_fuel_defaults":
            continue
        stabilized_eids.append(eid)

    table.append(
        [
            query,
            root_quant_count,
            tested_quant_count,
            tested_edit_count,
            len(stabilized_eids),
            len(ba.errored_edits),
            len(ba.passed_edits),
            len(ba.timeout_edits),
            len(ba.unknown_edits),
        ]
    )

df = pd.DataFrame(
    table,
    columns=[
        "query",
        "root_quants",
        "tested_quants",
        "tested_edits",
        "edits_stabilized",
        "edits_errored",
        "edits_passed",
        "edits_timeout",
        "edits_unknown",
    ],
)

In [ ]:
df["quants_test_rate"] = df["tested_quants"] / df["root_quants"] * 100
df.drop("tested_quants", axis=1, inplace=True)
df["error_rate"] = df["edits_errored"] / df["tested_edits"] * 100
df.drop("edits_errored", axis=1, inplace=True)
df["pass_rate"] = df["edits_passed"] / df["tested_edits"] * 100
df.drop("edits_passed", axis=1, inplace=True)
df["timeout_rate"] = df["edits_timeout"] / df["tested_edits"]  * 100
df.drop("edits_timeout", axis=1, inplace=True)
df["unknown_rate"] = df["edits_unknown"] / df["tested_edits"] * 100
df.drop("edits_unknown", axis=1, inplace=True)
print(tabulate(df, headers="keys", tablefmt="psql"))
# df.round(2).to_csv("debugger3_status.csv", index=False)